In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import b3d
import sys
sys.path.append(b3d.get_root_path())

from tests.sama4d.keypoint_tracking.keypoint_tracking_task import KeypointTrackingTask
from tests.sama4d.data_curation import get_cheezitbox_scene_loader
import rerun as rr

In [4]:
scene_spec = get_cheezitbox_scene_loader()
scene = scene_spec["feature_track_data_loader"]()

In [5]:
task = KeypointTrackingTask.task_from_feature_track_data(scene, n_frames=5)

In [10]:
rr.init(f'task--{scene_spec["scene_name"]}-2')
rr.connect("127.0.0.1:8812")
task.visualize_task()

In [12]:
from tests.sama4d.keypoint_tracking.patch_tracking_solver import AdamPatchTracker
task_spec = task.get_task_specification()
solver = AdamPatchTracker()
solution = solver.solve(task_spec)

In [16]:
solver.visualize_solver_state(task_spec)

In [9]:
task_spec["Xs_WC"]

Pose(position=Array([[0.15, 0.15, 0.  ],
       [0.15, 0.15, 0.  ],
       [0.15, 0.15, 0.  ],
       [0.15, 0.15, 0.  ],
       [0.15, 0.15, 0.  ],
       [0.15, 0.15, 0.  ],
       [0.15, 0.15, 0.  ],
       [0.15, 0.15, 0.  ],
       [0.15, 0.15, 0.  ],
       [0.15, 0.15, 0.  ],
       [0.15, 0.15, 0.  ],
       [0.15, 0.15, 0.  ],
       [0.15, 0.15, 0.  ],
       [0.15, 0.15, 0.  ],
       [0.15, 0.15, 0.  ],
       [0.15, 0.15, 0.  ],
       [0.15, 0.15, 0.  ],
       [0.15, 0.15, 0.  ],
       [0.15, 0.15, 0.  ],
       [0.15, 0.15, 0.  ],
       [0.15, 0.15, 0.  ],
       [0.15, 0.15, 0.  ],
       [0.15, 0.15, 0.  ],
       [0.15, 0.15, 0.  ],
       [0.15, 0.15, 0.  ],
       [0.15, 0.15, 0.  ],
       [0.15, 0.15, 0.  ],
       [0.15, 0.15, 0.  ],
       [0.15, 0.15, 0.  ],
       [0.15, 0.15, 0.  ]], dtype=float32), quaternion=Array([[ 0.27059808,  0.65328157, -0.65328157, -0.27059808],
       [ 0.27059808,  0.65328157, -0.65328157, -0.27059808],
       [ 0.27059808,  0.65

In [18]:
pos0_C = solver.Xs_CP_init.pos
quat0_C = solver.Xs_CP_init.xyzw

trace = solver.get_trace(pos0_C, quat0_C, task_spec["video"][0])

In [24]:
import jax
Xs_CO = trace["poses"].inner.value
patch_centers_C = jax.vmap(lambda X_WO: X_WO.pos)(Xs_CO)
patch_centers_C

Array([[-0.05000212, -0.06619699,  0.135087  ],
       [-0.02809205, -0.07648577,  0.15603575],
       [ 0.00310154, -0.07616813,  0.15538305],
       [ 0.03404703, -0.07586545,  0.1547727 ],
       [-0.06710628, -0.05293118,  0.17776427],
       [-0.03317447, -0.05524367,  0.18426888],
       [ 0.00367437, -0.05510603,  0.18383041],
       [ 0.04041119, -0.05506616,  0.18369837],
       [-0.07099946, -0.01869497,  0.1868448 ],
       [-0.03363629, -0.01869988,  0.18686473],
       [ 0.00372244, -0.01865589,  0.18640634],
       [ 0.04086833, -0.01859571,  0.18578853],
       [-0.06997868,  0.01839572,  0.18411441],
       [-0.03291819,  0.01825263,  0.18284887],
       [ 0.00364428,  0.01820436,  0.18237554],
       [ 0.04012314,  0.01821066,  0.18236308],
       [-0.06949101,  0.05482639,  0.18278907],
       [-0.03235891,  0.05388781,  0.1796875 ],
       [ 0.00358169,  0.05370409,  0.17908461],
       [ 0.03963393,  0.05400615,  0.18006866],
       [-0.06887942,  0.09054627,  0.181

In [34]:
import jax.numpy as jnp
import numpy as np
rr.log("tmp", rr.Points3D(positions=np.zeros((3)), radii=0.02), timeless=True)

In [26]:
patch_centers_C.shape

(24, 3)